Truy cập Links cửa hàng để lấy thông tin sản phẩm.

In [2]:
from time import sleep


In [8]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementClickInterceptedException


# Hàm cuộn trang có giới hạn (chừa lại 1/7 trang)
def scroll_page(driver, scroll_count=10, pause=1.5, offset=25):
    """Cuộn trang để tải thêm nội dung."""
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = (total_height - offset) / scroll_count
    for i in range(1, scroll_count + 1):
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")
        time.sleep(pause)
    time.sleep(2)  # Đợi 2 giây sau khi cuộn xong

# Hàm nhấn nút "Xem thêm" nếu còn
def click_view_more(driver):
    """Tìm và nhấn vào nút 'Xem Thêm' nếu có."""
    try:
        while True:
            view_more_button = driver.find_element(By.CSS_SELECTOR, "button.ViewMoreBtn__Wrapper-sc-qs9ydg-0")
            if view_more_button.is_displayed():
                driver.execute_script("arguments[0].click();", view_more_button)
                # print("Đã nhấn 'Xem Thêm'. Chờ tải thêm sản phẩm...")
                time.sleep(3)  # Đợi trang tải
            else:
                break
    except (NoSuchElementException, ElementClickInterceptedException):
        # print("Không tìm thấy hoặc không thể ấn nút 'Xem Thêm'.")
        return


# Đường dẫn file chứa danh sách cửa hàng
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links.xlsx"

# Đọc danh sách seller links
df = pd.read_excel(input_file)
seller_links = df["Seller Links"].dropna().tolist()[:1]

# Khởi tạo trình duyệt
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)  # Chờ tối đa 10 giây

# Danh sách lưu thông tin
data = []

# Duyệt từng cửa hàng
for link in seller_links:
    try:
        print(f"Đang truy cập: {link}")
        try:
            driver.get(link)
            time.sleep(3)  # Chờ trang tải
        except WebDriverException as e:
            print(f"Lỗi khi mở trang {link}: {e}")
            continue

        # Bước 1: Lấy tên cửa hàng
        try:
            # store_name_element = driver.find_element(By.CSS_SELECTOR, "h1.sc-b4c2fa32-2.fXWGFK")
            store_name_element = driver.find_element(By.CSS_SELECTOR, "h1.Header__SellerName-sc-vgcg69-2.kgqRrB")
            store_name = store_name_element.text.strip()
            print(f"Tên cửa hàng: {store_name}")
        except NoSuchElementException:
            store_name = None
            print("Không tìm thấy tên cửa hàng.")


        # Bước 2: Click vào tab "Hồ Sơ Cửa Hàng"
        try:
            profile_tab = driver.find_element(By.XPATH, "//div[contains(text(), 'Hồ Sơ Cửa Hàng')]")
            profile_tab.click()
            time.sleep(3)  # Chờ trang tải lại sau khi click
            print("Đã click vào tab 'Hồ Sơ Cửa Hàng'")
        except NoSuchElementException:
            print("Không tìm thấy tab 'Hồ Sơ Cửa Hàng', bỏ qua trang này.")
            continue

        # Bước 3: Lấy thông tin cửa hàng
        try:
            # info_values = driver.find_elements(By.CSS_SELECTOR, 
            #     "span.sc-ba05c94e-5.igQJxN, "
            #     "span.sc-ba05c94e-4.fAQpWx")
            info_values = driver.find_elements(By.CSS_SELECTOR, 
                "span.StoreInfo__InfoValue-sc-1un24du-5.GxEKI, "
                "span.StoreInfo__InfoLabel-sc-1un24du-4.jpeWpY")

            extracted_values = [value.text.strip() for value in info_values if value.text.strip()]
            # print(f"Thông tin cửa hàng: {extracted_values}")

            year = extracted_values[2]
            total_products = extracted_values[5] 
            rating = extracted_values[11][:3]
            review_count = extracted_values[12]
            followers = extracted_values[15] 
            chat_response = extracted_values[18]
            print(year, total_products, rating, review_count, followers, chat_response)
            
        except Exception as e:
            print(f"Lỗi khi lấy thông tin cửa hàng {link}: {e}")
            year, total_products, rating, review_count, followers, chat_response = (None, None, None, None, None, None)


        # Bước 4: Click vào tab "Tất Cả Sản Phẩm"
        try:
            all_products_tab = driver.find_element(By.XPATH, "//div[contains(text(), 'Tất Cả Sản Phẩm')]")
            all_products_tab.click()
            time.sleep(3)  # Chờ trang tải lại sau khi click
            # print("Đã click vào tab 'Tất Cả Sản Phẩm'")
        except NoSuchElementException:
            print("Không tìm thấy tab 'Tất Cả Sản Phẩm', bỏ qua sản phẩm.")
            continue


        # Bước 5: Cuộn trang và nhấn "Xem thêm"
        scroll_page(driver)
        # print("Đã cuộn trang. Đang lấy thông tin sản phẩm...")
        click_view_more(driver)
        # print("Đã click viewmore.")

        # Bước 4: Lấy thông tin từng sản phẩm (Tên, Giá, Số lượng bán)
        try:
            # product_elements = driver.find_elements(By.CSS_SELECTOR, "a.sc-e14980c3-2.sc-e14980c3-3.hQMtFc")
            product_elements = driver.find_elements(By.CSS_SELECTOR, "a.Product__Wrapper-sc-n99tp2-2.Product___StyledWrapper-sc-n99tp2-3.fNzCgC")

            for product in product_elements:
                try:
                    # Lấy tên sản phẩm (nằm trong thẻ <h3> bên trong div chứa class "kuTTXp")
                    # product_name = product.find_element(By.CSS_SELECTOR, "div.sc-e14980c3-7.kuTTXp > h3").text.strip()
                    product_name = product.find_element(By.CSS_SELECTOR, "div.Product__Name-sc-n99tp2-7.cfPcgv > h3").text.strip()
                    print(product_name)
                except Exception as e:
                    product_name = None
                    print("Không lấy được tên sản phẩm:", e)

                # Lấy href (đường dẫn chi tiết sản phẩm từ thuộc tính href của thẻ <a>)
                product_link = product.get_attribute("href")
                print(product_link)
                
                try:
                    # Lấy rating: giá trị rating nằm trong div có class "sc-e14980c3-25 iUUFFW"
                    # product_rating = product.find_element(By.CSS_SELECTOR, "div.sc-e14980c3-25.iUUFFW").text.strip()
                    product_rating = product.find_element(By.CSS_SELECTOR, "div.Product__TextRating-sc-n99tp2-25.dDxtAi").text.strip()
                    print(product_rating)
                except Exception as e:
                    product_rating = None
                    print("Không lấy được rating:", e)

                try:
                    # Lấy số lượng đã bán: nằm trong thẻ <span class="percent">
                    sold_text = product.find_element(By.CSS_SELECTOR, "span.percent").text.strip()
                    # Loại bỏ chữ "Đã bán " nếu có
                    product_quantity_sold = sold_text.replace("Đã bán ", "")
                    print(product_quantity_sold)
                except Exception as e:
                    product_quantity_sold = None
                    print("Không lấy được số lượng đã bán:", e)
                
                try:
                    # Lấy giá sản phẩm: giá nằm trong thẻ <span> bên trong div có class "cspIwy has-discount"
                    product_price = product.find_element(By.CSS_SELECTOR, "div.Product__PriceDiscountValue-sc-n99tp2-9.eWvHOp span").text.strip()
                    print(product_price)
                except Exception as e:                                         
                    product_price = None
                    print("Không lấy được giá sản phẩm:", e)




                # Lưu dữ liệu vào danh sách
                data.append({
                    "Seller Link": link,
                    "Store Name": store_name,
                    "Year Joined": year,
                    "Total Products": total_products,
                    "Rating": rating,
                    "Review Count": review_count,
                    "Followers": followers,
                    "Chat Response Rate": chat_response,
                    "Product Name": product_name,
                    "Product Link": product_link,
                    "Product Rating": product_rating,
                    "Price (VND)": product_price,
                    "Sold Quantity": product_quantity_sold
                })

            print(data)
        except Exception as e:
            print(f"Lỗi khi lấy danh sách sản phẩm tại {link}: {e}")
            continue

    except Exception as e:
        print(f"Lỗi khi xử lý {link}: {e}")

# Đóng trình duyệt
driver.quit()

# Lưu vào Excel
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_products.xlsx"
output_df = pd.DataFrame(data)
output_df.to_excel(output_file, index=False)

print(f"\nĐã lưu thông tin vào file: {output_file}")

Đang truy cập: https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic
Tên cửa hàng: Công Nghệ Việt
Đã click vào tab 'Hồ Sơ Cửa Hàng'
2017 66+ 4.4 (21k+) 1.7k+ 50%
Cáp sạc Baseus Cafule sạc nhanh dùng cho iPhone - Hàng chính hãng
https://tiki.vn/cap-sac-baseus-cafule-sac-nhanh-dung-cho-iphone-hang-chinh-hang-p142439870.html?spid=142439872
4.8
3257
75.000
Tai Nghe Bluetooth True Wireless AMOI F9 Cảm Ứng Vân Tay, Nâng Cấp Dock Sạc có Led Báo Pin Kép - Hàng Nhập Khẩu
https://tiki.vn/tai-nghe-bluetooth-true-wireless-amoi-f9-5-0-cam-ung-van-tay-nang-cap-dock-sac-co-led-bao-pin-kep-hang-nhap-khau-p191684818.html?spid=210637817
4.5
3654
89.000
Tai Nghe Nhét Tai Langsdom Super Bass JM26
https://tiki.vn/tai-nghe-nhet-tai-langsdom-super-bass-jm26-hang-nhap-khau-p895314.html?spid=899508
4.2
10649
49.000
Tai nghe chuyên game có Mic Plextone G25
https://tiki.vn/tai-nghe-chuyen-game-co-mic-plextone-g25-hang-chinh-hang-p10784603.html?spid=10784607
4.5
1386
184.000
B

In [9]:
import pandas as pd

# Thiết lập để hiển thị tất cả các cột
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)  # Đảm bảo chiều rộng hiển thị không bị giới hạn
pd.set_option('display.max_colwidth', None)  # Hiển thị đầy đủ nội dung trong mỗi cột

df_new = pd.DataFrame(data)
display(df_new)


,Seller Link,Store Name,Year Joined,Total Products,Rating,Review Count,Followers,Chat Response Rate,Product Name,Product Link,Product Rating,Price (VND),Sold Quantity
0,https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic,Công Nghệ Việt,2017,66+,4.4,(21k+),1.7k+,50%,Cáp sạc Baseus Cafule sạc nhanh dùng cho iPhone - Hàng chính hãng,https://tiki.vn/cap-sac-baseus-cafule-sac-nhanh-dung-cho-iphone-hang-chinh-hang-p142439870.html?spid=142439872,4.8,75.000,3257
1,https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic,Công Nghệ Việt,2017,66+,4.4,(21k+),1.7k+,50%,"Tai Nghe Bluetooth True Wireless AMOI F9 Cảm Ứng Vân Tay, Nâng Cấp Dock Sạc có Led Báo Pin Kép - Hàng Nhập Khẩu",https://tiki.vn/tai-nghe-bluetooth-true-wireless-amoi-f9-5-0-cam-ung-van-tay-nang-cap-dock-sac-co-led-bao-pin-kep-hang-nhap-khau-p191684818.html?spid=210637817,4.5,89.000,3654
2,https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic,Công Nghệ Việt,2017,66+,4.4,(21k+),1.7k+,50%,Tai Nghe Nhét Tai Langsdom Super Bass JM26,https://tiki.vn/tai-nghe-nhet-tai-langsdom-super-bass-jm26-hang-nhap-khau-p895314.html?spid=899508,4.2,49.000,10649
3,https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic,Công Nghệ Việt,2017,66+,4.4,(21k+),1.7k+,50%,Tai nghe chuyên game có Mic Plextone G25,https://tiki.vn/tai-nghe-chuyen-game-co-mic-plextone-g25-hang-chinh-hang-p10784603.html?spid=10784607,4.5,184.000,1386
4,https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic,Công Nghệ Việt,2017,66+,4.4,(21k+),1.7k+,50%,"Bơm xe đạp, xe máy xách tay siêu nhẹ, áp suất 120 PSI, thân hợp kim siêu bền, hỗ trợ cả van Mỹ và van Pháp, bơm được cho tất cả các loại xe đạp bao gồm xe đua, xe địa hình, xe đạp đường trường, thương hiệu Wheel Up - Hàng chính hãng",https://tiki.vn/bom-xe-dap-xach-tay-sieu-nhe-ap-suat-120-psi-than-hop-kim-sieu-ben-ho-tro-ca-van-my-va-van-phap-bom-duoc-cho-tat-ca-cac-loai-xe-dap-bao-gom-xe-dua-xe-dia-hinh-xe-dap-duong-truong-thuong-hieu-wheel-up-hang-chinh-hang-p50819880.html?spid=74529421,4.6,95.000,2300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic,Công Nghệ Việt,2017,66+,4.4,(21k+),1.7k+,50%,Đèn để bàn thông minh Baseus Smart Eye Series Charging Folding Reading Desk Lamp - LVP007 [Hàng Chính Hãng,https://tiki.vn/den-de-ban-thong-minh-baseus-smart-eye-series-charging-folding-reading-desk-lamp-lvp007-hang-chinh-hang-p108433041.html?spid=275232737,None,629.000,7
62,https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic,Công Nghệ Việt,2017,66+,4.4,(21k+),1.7k+,50%,"Cóc củ sạc nhanh 24W trang bị Quick Charger 3.0 hiệu Baseus Wireless Charging Quick Charger (Sạc nhanh QC 3.0, công suất 24W, trang bị chip sạc thông minh) - Hàng nhập khẩu",https://tiki.vn/coc-cu-sac-nhanh-24w-trang-bi-quick-charger-3-0-hieu-baseus-wireless-charging-quick-charger-sac-nhanh-qc-3-0-cong-suat-24w-trang-bi-chip-sac-thong-minh-hang-nhap-khau-p52874490.html?spid=275125290,None,148.000,None
63,https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic,Công Nghệ Việt,2017,66+,4.4,(21k+),1.7k+,50%,"Cốc sạc nhanh siêu nhỏ gọn chân tròn Baseus Compact Quick Charger 20W (USB + Type C Dual Port, 20W PD/QC 3.0 Multi Quick Charge Support) dùng cho Samsung / Xiaomi/ Huawei/ OPPO / iPhone/iPad......Hàng chính hãng",https://tiki.vn/coc-sac-nhanh-sieu-nho-gon-chan-tron-baseus-compact-quick-charger-20w-usb-type-c-dual-port-20w-pd-qc-3-0-multi-quick-charge-support-dung-cho-samsung-xiaomi-huawei-oppo-iphone-ipad-hang-chinh-hang-p157244873.html?spid=275008559,None,218.000,2
64,https://tiki.vn/cua-hang/cong-nghe-viet?source_screen=product_detail&source_engine=organic,Công Nghệ Việt,2017,66+,4.4,(21k+),1.7k+,50%,Cáp sạc nhanh PD20W truyền dữ liệu Baseus Metal Data Type-C s

Truy cập vào từng sản phẩm trong store

In [23]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException

# Đọc file Excel chứa thông tin sản phẩm
file_path = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_products.xlsx"

# Đọc dữ liệu từ file gốc
df = pd.read_excel(file_path)

# Kiểm tra xem cột "Product Link" có tồn tại không
if "Product Link" not in df.columns:
    raise ValueError("Cột 'Product Link' không tồn tại trong file Excel.")

# Chọn phạm vi sản phẩm cần lấy dữ liệu
product_links = df["Product Link"].dropna().tolist()[43:47]  # Chạy từ sản phẩm 10-20

# Khởi tạo trình duyệt Selenium
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)

# Tạo các cột mới nếu chưa có
new_columns = ["Overall Rating", "Total Reviews", "5 Stars", "4 Stars", "3 Stars", "2 Stars", "1 Star"]
for col in new_columns:
    if col not in df.columns:
        df[col] = None

# Duyệt qua từng sản phẩm trong phạm vi đã chọn
for link in product_links:
    try:
        # Tìm vị trí hàng trong DataFrame dựa trên "Product Link"
        row_index = df[df["Product Link"] == link].index[0]

        driver.get(link)
        time.sleep(3)  # Chờ trang tải

        # Cuộn xuống 3 lần trước khi tới giữa trang
        total_height = driver.execute_script("return document.body.scrollHeight")
        for i in range(1, 2):
            driver.execute_script(f"window.scrollTo(0, {i * (total_height / 2.8)});")
            time.sleep(1)  # Chờ trang tải nội dung

        # Lấy tổng điểm đánh giá
        try:
            overall_rating = driver.find_element(By.CSS_SELECTOR, "div.review-rating__point").text.strip()
        except NoSuchElementException:
            overall_rating = None

        # Lấy tổng số đánh giá
        try:
            total_reviews = driver.find_element(By.CSS_SELECTOR, "div.review-rating__total").text.strip()
            total_reviews = total_reviews.replace("(", "").replace(" đánh giá)", "")
        except NoSuchElementException:
            total_reviews = None

        # Lấy số lượng đánh giá theo sao
        ratings_count = {}
        for i in range(5, 0, -1):
            try:
                rating_xpath = f"//div[@class='review-rating__level'][{6 - i}]/div[@class='review-rating__number']"
                ratings_count[f"{i}_stars"] = driver.find_element(By.XPATH, rating_xpath).text.strip()
            except NoSuchElementException:
                ratings_count[f"{i}_stars"] = None

        # Cập nhật vào DataFrame tại vị trí tìm thấy
        df.at[row_index, "Overall Rating"] = overall_rating
        df.at[row_index, "Total Reviews"] = total_reviews
        df.at[row_index, "5 Stars"] = ratings_count["5_stars"]
        df.at[row_index, "4 Stars"] = ratings_count["4_stars"]
        df.at[row_index, "3 Stars"] = ratings_count["3_stars"]
        df.at[row_index, "2 Stars"] = ratings_count["2_stars"]
        df.at[row_index, "1 Star"] = ratings_count["1_stars"]

        print(f"✅ Đã cập nhật dữ liệu cho sản phẩm: {link}")

    except WebDriverException as e:
        print(f"❌ Lỗi khi truy cập {link}: {e}")
        continue

# Đóng trình duyệt
driver.quit()

# Ghi dữ liệu vào cùng file mà không ghi đè dữ liệu cũ
df.to_excel(file_path, index=False)

print(f"\n📌 Dữ liệu mới đã được thêm vào file: {file_path}")


C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_21712\782845663.py:69: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '5.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[row_index, "Overall Rating"] = overall_rating
C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_21712\782845663.py:70: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[row_index, "Total Reviews"] = total_reviews
C:\Users\CUONG DAT PHAT\AppData\Local\Temp\ipykernel_21712\782845663.py:71: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
 

✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/tai-nghe-true-wireless-sony-wf-1000xm4-chong-on-chu-dong-hang-chinh-hang-p109267785.html?spid=129481173
✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/plextone-t100-cap-sac-da-nang-type-c-toc-do-nhanh-do-ben-vuot-troi-co-den-bao-hang-chinh-hang-p274883232.html?spid=274883234
✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/cap-sac-baseus-superior-series-sac-nhanh-dung-cho-iphone-2-4a-480mbps-fast-charge-abs-tpe-cable-hang-chinh-hang-p143362873.html?spid=198739917
✅ Đã cập nhật dữ liệu cho sản phẩm: https://tiki.vn/tai-nghe-true-wireless-earbuds-soundpeats-mac-thoi-gian-su-dung-60-gio-chong-nuoc-ipx7-bluetooth-v5-0-hang-chinh-hang-p75801356.html?spid=198240754

📌 Dữ liệu mới đã được thêm vào file: D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_products.xlsx


In [3]:
import pandas as pd
import threading
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook

# Đường dẫn file Excel
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_links.xlsx"
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links_2.xlsx"
error_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/Still not collected seller yet_2.xlsx"

# Đọc file Excel và lấy 20 dòng đầu tiên
df = pd.read_excel(input_file)
links = df["Links"].tolist()[10400:10700]

# Sử dụng tập hợp để tránh trùng lặp seller links
seller_links_set = set()

# Danh sách lưu các URL bị lỗi
error_links = []

# Khóa để đảm bảo ghi file an toàn khi dùng đa luồng
lock = threading.Lock()

# Hàm xử lý từng link
def process_link(url):
    global seller_links_set, error_links

    # Khởi tạo Selenium WebDriver trong từng luồng
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.implicitly_wait(10)

    try:
        print(f"🔹 Đang truy cập: {url}")
        driver.get(url)

        # **Thực hiện reload trang sau khi truy cập**
        driver.refresh()

        # Chờ trang load hoàn toàn sau khi reload (timeout 10s)
        WebDriverWait(driver, 10).until(
            # EC.presence_of_element_located((By.CSS_SELECTOR, "span.seller-name div.sc-d1e96cea-0.jBNFDc a"))
            EC.presence_of_element_located((By.CSS_SELECTOR, "span.seller-name div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a"))

        )

        # Tìm seller link
        # seller_element = driver.find_element(By.CSS_SELECTOR, "span.seller-name div.sc-d1e96cea-0.jBNFDc a")
        seller_element = driver.find_element(By.CSS_SELECTOR, "span.seller-name div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a")

        seller_href = seller_element.get_attribute("href")
        print(f"  ✅ Seller link: {seller_href}")

        # Dùng khóa để đảm bảo an toàn khi ghi file
        with lock:
            if seller_href not in seller_links_set:
                seller_links_set.add(seller_href)

                # Đọc file hiện tại (nếu có)
                try:
                    existing_df = pd.read_excel(output_file)
                except FileNotFoundError:
                    existing_df = pd.DataFrame(columns=["Seller Links"])

                # Thêm dữ liệu mới
                new_df = pd.DataFrame({"Seller Links": [seller_href]})
                final_df = pd.concat([existing_df, new_df], ignore_index=True)

                # Ghi vào file Excel
                with pd.ExcelWriter(output_file, mode="w", engine="openpyxl") as writer:
                    final_df.to_excel(writer, index=False)

    except TimeoutException:
        print(f"  ⚠️ Trang {url} load quá lâu, bỏ qua.")
        save_error(url, "Load quá lâu")
    except NoSuchElementException:
        print(f"  ❌ Không tìm thấy seller link trên trang: {url}")
        save_error(url, "Không tìm thấy seller link")
    except Exception as e:
        print(f"  ❌ Lỗi khi xử lý {url}: {e}")
        save_error(url, str(e))
    finally:
        driver.quit()

# Hàm lưu các URL bị lỗi vào file "Still not collected seller yet.xlsx"
def save_error(url, error_message):
    with lock:
        try:
            existing_error_df = pd.read_excel(error_file)
        except FileNotFoundError:
            existing_error_df = pd.DataFrame(columns=["URL", "Lỗi"])

        new_error_df = pd.DataFrame({"URL": [url], "Lỗi": [error_message]})
        final_error_df = pd.concat([existing_error_df, new_error_df], ignore_index=True)

        with pd.ExcelWriter(error_file, mode="w", engine="openpyxl") as writer:
            final_error_df.to_excel(writer, index=False)

# Sử dụng ThreadPoolExecutor để chạy đa luồng
max_threads = 3  # Số luồng tối đa có thể điều chỉnh
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    executor.map(process_link, links)

print(f"\n✅ Đã hoàn thành. Dữ liệu lỗi được lưu vào: {error_file}")


🔹 Đang truy cập: https://tiki.vn/tai-nghe-dan-khi-qua-xuong-cao-cap-teementtai-nghe-bluetooth-khong-day-danh-cho-the-thao-va-chay-bo-hang-chinh-hang-p275941085.html?spid=275941089🔹 Đang truy cập: https://tiki.vn/gap-toc-thoi-trang-cao-cap-nu-p93278471.html?spid=93278472

🔹 Đang truy cập: https://tiki.vn/quat-tan-nhiet-coolmoon-v2-rgb-hang-nhap-khau-p23701634.html?spid=56104782
  ✅ Seller link: https://tiki.vn/cua-hang/venie?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://tiki.vn/cua-hang/shopducthang?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://tiki.vn/cua-hang/extreme-zero-online?source_screen=product_detail&source_engine=organic
🔹 Đang truy cập: https://tiki.vn/vi-nu-vai-polyester-kantapro-dang-ngang-sol-wallet-5-colors-p204117014.html?spid=204117024
🔹 Đang truy cập: https://tiki.vn/tui-deo-cheo-classic-black-vns-vn8005-tui-deo-cheo-cao-cap-chong-tham-nuoc-24-5cm-x-9cm-x-14cm-classic-crossbody-bag-p77560504.html?spid=77560505

In [7]:
import pandas as pd
import threading
from concurrent.futures import ThreadPoolExecutor
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import load_workbook

# Đường dẫn file Excel
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_links.xlsx"
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/store_seller_links_10.xlsx"
error_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/Still not collected seller yet_10.xlsx"

# Đọc file Excel và lấy 20 dòng đầu tiên
df = pd.read_excel(input_file)
links = df["Links"].tolist()[12800:13100]

# Sử dụng tập hợp để tránh trùng lặp seller links
seller_links_set = set()

# Danh sách lưu các URL bị lỗi
error_links = []

# Khóa để đảm bảo ghi file an toàn khi dùng đa luồng
lock = threading.Lock()

# Hàm xử lý từng link
def process_link(url):
    global seller_links_set, error_links

    # Khởi tạo Selenium WebDriver trong từng luồng
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.implicitly_wait(10)

    try:
        print(f"🔹 Đang truy cập: {url}")
        driver.get(url)

        # **Thực hiện reload trang sau khi truy cập**
        driver.refresh()

        # Chờ trang load hoàn toàn sau khi reload (timeout 10s)
        WebDriverWait(driver, 10).until(
            # EC.presence_of_element_located((By.CSS_SELECTOR, "span.seller-name div.sc-d1e96cea-0.jBNFDc a"))
            EC.presence_of_element_located((By.CSS_SELECTOR, "span.seller-name div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a"))

        )

        # Tìm seller link
        # seller_element = driver.find_element(By.CSS_SELECTOR, "span.seller-name div.sc-d1e96cea-0.jBNFDc a")
        seller_element = driver.find_element(By.CSS_SELECTOR, "span.seller-name div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a")

        seller_href = seller_element.get_attribute("href")
        print(f"  ✅ Seller link: {seller_href}")

        # Dùng khóa để đảm bảo an toàn khi ghi file
        with lock:
            if seller_href not in seller_links_set:
                seller_links_set.add(seller_href)

                # Đọc file hiện tại (nếu có)
                try:
                    existing_df = pd.read_excel(output_file)
                except FileNotFoundError:
                    existing_df = pd.DataFrame(columns=["Seller Links"])

                # Thêm dữ liệu mới
                new_df = pd.DataFrame({"Seller Links": [seller_href]})
                final_df = pd.concat([existing_df, new_df], ignore_index=True)

                # Ghi vào file Excel
                with pd.ExcelWriter(output_file, mode="w", engine="openpyxl") as writer:
                    final_df.to_excel(writer, index=False)

    except TimeoutException:
        print(f"  ⚠️ Trang {url} load quá lâu, bỏ qua.")
        save_error(url, "Load quá lâu")
    except NoSuchElementException:
        print(f"  ❌ Không tìm thấy seller link trên trang: {url}")
        save_error(url, "Không tìm thấy seller link")
    except Exception as e:
        print(f"  ❌ Lỗi khi xử lý {url}: {e}")
        save_error(url, str(e))
    finally:
        driver.quit()

# Hàm lưu các URL bị lỗi vào file "Still not collected seller yet.xlsx"
def save_error(url, error_message):
    with lock:
        try:
            existing_error_df = pd.read_excel(error_file)
        except FileNotFoundError:
            existing_error_df = pd.DataFrame(columns=["URL", "Lỗi"])

        new_error_df = pd.DataFrame({"URL": [url], "Lỗi": [error_message]})
        final_error_df = pd.concat([existing_error_df, new_error_df], ignore_index=True)

        with pd.ExcelWriter(error_file, mode="w", engine="openpyxl") as writer:
            final_error_df.to_excel(writer, index=False)

# Sử dụng ThreadPoolExecutor để chạy đa luồng
max_threads = 3  # Số luồng tối đa có thể điều chỉnh
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    executor.map(process_link, links)

print(f"\n✅ Đã hoàn thành. Dữ liệu lỗi được lưu vào: {error_file}")


🔹 Đang truy cập: https://tiki.vn/tui-xe-dap-gan-baga-sau-xe-dung-hanh-ly-zz04-chat-lieu-vai-polyester-840d-khoa-zip-doi-chong-nuoc-dung-tich-lon-mai-lee-p247413145.html?spid=247413146
🔹 Đang truy cập: https://tiki.vn/vi-da-nam-gence-bop-nam-da-bo-epsom-mau-den-cao-cap-c09-dang-doc-bao-hanh-chinh-hang-p204988337.html?spid=204988338
🔹 Đang truy cập: https://tiki.vn/tui-dung-do-my-pham-cotton-tien-dung-205880-p116296335.html?spid=116296353
  ✅ Seller link: https://tiki.vn/cua-hang/gence-hai-phong?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://tiki.vn/cua-hang/do-da-thanh-long-tlg?source_screen=product_detail&source_engine=organic
  ✅ Seller link: https://tiki.vn/cua-hang/mai-lee-house?source_screen=product_detail&source_engine=organic
🔹 Đang truy cập: https://tiki.vn/tay-moc-tuong-moc-quan-ao-dam-dac-ht01-5lit-dung-cho-nha-nghi-khach-san-giat-la-ve-sinh-cong-nghiep-p52393121.html?spid=52393122
🔹 Đang truy cập: https://tiki.vn/nuoc-yen-sao-win-snest-junior-12-4